<a href="https://colab.research.google.com/github/RVijayChiranjithReddy/Autolabel_encoder/blob/main/ALE_WCYCLEGAN_Multilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 686kB 8.1MB/s 


In [ ]:

from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.initializers import RandomNormal
from keras.applications import *
import tensorflow as tf
from keras.models import Sequential,Model
import matplotlib.pyplot as plt
import keras.backend as K
from keras.optimizers import RMSprop, SGD, Adam
import cv2
from PIL import Image
import numpy as np
import tensorflow_addons as tfa
import glob
from random import randint, shuffle
%matplotlib inline
K.set_learning_phase(1)


/usr/local/lib/python3.7/dist-packages/keras/backend.py:400: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
df = pandas.read_csv('/content/styles.csv')
df.fillna('DK', inplace = True) 
data = {}
for i in range(44446):
  tags = [str(x) for x in df.loc[i].values[1:9] if type(x) is str]
  #desc = df.loc[i].values[9]
  #if type(desc) is not float:
    #for de in desc.split():
     #if de.isalpha() and de not in tags:
        #tags.append(de)
  data[str(df.loc[i].values[0])] = ':'.join(tags)
vocab = []
for k,v in data.items():
  temp = list(set(v.split(':')))
  vocab = vocab + temp
  vocab = list(set(vocab))
print(len(vocab))
#from tensorflow.keras.preprocessing.text import one_hot
#for k,d in data.items():
  #data_encoded[k] = one_hot(' '.join(d),len(vocab))
tokenizer1 = Tokenizer(num_words=25)
tokenizer2 = Tokenizer(num_words=25)
tokenizer3 = Tokenizer(num_words=54)
tokenizer4 = Tokenizer(num_words=173)
tokenizer5 = Tokenizer(num_words=48)
tokenizer6 = Tokenizer(num_words=25)
tokenizer7 = Tokenizer(num_words=25)
# This builds the word index
tokenizer1.fit_on_texts(df['gender'])
tokenizer2.fit_on_texts(df['masterCategory'])
tokenizer3.fit_on_texts(df['subCategory'])
tokenizer4.fit_on_texts(df['articleType'])
tokenizer5.fit_on_texts(df['baseColour'])
tokenizer6.fit_on_texts(df['season'])
tokenizer7.fit_on_texts(df['usage'])


# This turns strings into lists of integer indices.
#train_sequences = tokenizer.texts_to_sequences(data.values())
print(tokenizer1.word_index)
print(tokenizer2.word_index)
print(tokenizer3.word_index)
print(tokenizer4.word_index)
print(tokenizer5.word_index)
print(tokenizer6.word_index)
print(tokenizer7.word_index)

240
{'men': 1, 'women': 2, 'unisex': 3, 'boys': 4, 'girls': 5}
{'apparel': 1, 'accessories': 2, 'footwear': 3, 'personal': 4, 'care': 5, 'free': 6, 'items': 7, 'sporting': 8, 'goods': 9, 'home': 10}
{'topwear': 1, 'shoes': 2, 'bags': 3, 'bottomwear': 4, 'watches': 5, 'innerwear': 6, 'jewellery': 7, 'eyewear': 8, 'fragrance': 9, 'sandal': 10, 'wallets': 11, 'flip': 12, 'flops': 13, 'belts': 14, 'socks': 15, 'lips': 16, 'and': 17, 'dress': 18, 'loungewear': 19, 'nightwear': 20, 'saree': 21, 'nails': 22, 'makeup': 23, 'headwear': 24, 'ties': 25, 'accessories': 26, 'skin': 27, 'scarves': 28, 'cufflinks': 29, 'apparel': 30, 'set': 31, 'free': 32, 'gifts': 33, 'stoles': 34, 'care': 35, 'eyes': 36, 'mufflers': 37, 'shoe': 38, 'sports': 39, 'equipment': 40, 'gloves': 41, 'hair': 42, 'bath': 43, 'body': 44, 'water': 45, 'bottle': 46, 'perfumes': 47, 'umbrellas': 48, 'beauty': 49, 'wristbands': 50, 'home': 51, 'furnishing': 52, 'vouchers': 53}
{'tshirts': 1, 'shoes': 2, 'shirts': 3, 'casual': 4,

In [ ]:
da = {}
ds = {}
for k,v in data.items():
  da[str(k)] =[]
  da[str(k)].append(tokenizer1.texts_to_sequences([v.split(':')[0]])[0][0])
  da[str(k)].append(tokenizer2.texts_to_sequences([v.split(':')[1]])[0][0])
  da[str(k)].append(tokenizer3.texts_to_sequences([v.split(':')[2]])[0][0])
  da[str(k)].append(tokenizer4.texts_to_sequences([v.split(':')[3]])[0][0])
  da[str(k)].append(tokenizer5.texts_to_sequences([v.split(':')[4]])[0][0])
  da[str(k)].append(tokenizer6.texts_to_sequences([v.split(':')[5]])[0][0])
  da[str(k)].append(tokenizer7.texts_to_sequences([v.split(':')[6]])[0][0])
  i=i+1


In [ ]:
def read_image(fn):
  im = cv2.imread(fn)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
  im = cv2.resize(im, (128,128), interpolation=cv2.INTER_CUBIC)
  im = np.array(im)/255
  return im

In [ ]:
traini = glob.glob(r'/content/drive/MyDrive/images/*')
print(len(traini))
def minibatch(data, batchsize):
    length = len(data)
    epoch = i = 0
    tmpsize = None    
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1        
        rtn = [read_image(data[j]) for j in range(i,i+size)]
        ltn=[]
        se = []
        for j in range(i,i+size):
          if '(' in data[j].split('/')[-1].split('.')[0]:
            t=[]
            p = data[j].split('/')[-1].split('.')[0].split(' ')[0]
            t.append(da[p][0]/5)
            t.append(da[p][1]/10)
            t.append(da[p][2]/53)
            t.append(da[p][3]/172)
            t.append(da[p][4]/47)
            t.append(da[p][5]/5)
            t.append(da[p][6]/9)
            se.append(t) 
          else:
            t=[]
            p = data[j].split('/')[-1].split('.')[0]
            t.append(da[p][0]/5)
            t.append(da[p][1]/10)
            t.append(da[p][2]/53)
            t.append(da[p][3]/172)
            t.append(da[p][4]/47)
            t.append(da[p][5]/5)
            t.append(da[p][6]/9)
            se.append(t) 
        #ltn = [data_enc_padded[data[j].split('/')[-1].split('.')[0]] for j in range(i,i+size)]
        i+=size
        tmpsize = yield epoch, np.float32(rtn), np.array(se)

def minibatchAB(dataA, batchsize):
    batchA = minibatch(dataA, batchsize)
    tmpsize = None    
    while True:        
        ep1, A ,B = batchA.send(tmpsize)
        tmpsize = yield ep1, A , B
train_batch = minibatchAB(traini, 64)

44462


In [ ]:
class ClipConstraint(tf.keras.constraints.Constraint):
  def __init__(self, clip_value):
    self.clip_value = clip_value
  def __call__(self, weights):
    return backend.clip(weights, -self.clip_value, self.clip_value)
  def get_config(self):
    return {'clip_value': self.clip_value}
const = ClipConstraint(0.01)

In [ ]:
class CycleGAN():
  def __init__(self,train_batch,const):
    self.img_rows = 128 
    self.img_cols = 128 
    self.channels = 3 
    self.dloss = []
    self.gloss = []
    self.iter = []
    self.traindata = train_batch
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.label_shape = (7,)
    self.lambda_cycle = 10
    self.bloss = tf.keras.losses.binary_crossentropy
    optimizer = Adam(learning_rate = 0.002)
    self.const = const
    self.g_AB = self.build_label_generator() 
    self.g_BA = self.build_img_generator()
    A = Input(shape=self.img_shape) 
    B = Input(shape=self.label_shape) 
    f_B = self.g_AB(A) 
    fake_B = concatenate(f_B,axis=1)
    fake_A = self.g_BA(B)
    reconstr_A = self.g_BA(fake_B) 
    r_B = self.g_AB(fake_A)
    reconstr_B = concatenate(r_B,axis=1)
    self.d_A = self.build_img_label_discriminator() 
    self.d_B = self.build_img_label_discriminator() 
    self.d_A.compile(loss=self.wa_loss,optimizer=RMSprop(0.00005)) 
    self.d_B.compile(loss=self.wa_loss,optimizer=RMSprop(0.00005))  
    self.d_A.trainable = False 
    self.d_B.trainable = False
    valid_A = self.d_A([fake_A,B]) 
    valid_B = self.d_B([A,fake_B])
    self.classifier = Model(inputs = A,outputs = f_B)
    self.classifier.compile(loss = [self.bloss,self.bloss,self.bloss,self.bloss,self.bloss,self.bloss,self.bloss],optimizer = RMSprop(0.00005))
    self.combined = Model(inputs=[A,B],outputs=[valid_A,valid_B,reconstr_A]+r_B) 
    self.combined.compile(loss=[self.wa_loss,self.wa_loss,tf.keras.losses.binary_crossentropy,self.bloss,self.bloss,self.bloss,self.bloss,self.bloss,self.bloss,self.bloss],loss_weights=[1, 1,self.lambda_cycle, self.lambda_cycle],optimizer=RMSprop(0.00005))
  def wa_loss(self,y_true, y_pred):
    return K.mean(y_true * y_pred)
  def build_img_generator(self):
    d0 = Input(shape=self.label_shape)
    d1 = Dense(128,activation = 'relu')(d0)
    d2 = tf.keras.layers.Reshape((1,1,128))(d1)
    d3 = Conv2DTranspose(128,kernel_size = 3,strides = 2,padding = 'same')(d2)
    #d3 = BatchNormalization()(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(128,kernel_size = 3,strides = 1,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(128,kernel_size = 3,strides = 1,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(64,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(64,kernel_size = 3,strides = 1,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(64,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(64,kernel_size = 3,strides = 1,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(32,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = Conv2DTranspose(32,kernel_size = 3,strides = 1,padding = 'same')(d3)
    d3 = Conv2DTranspose(32,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(16,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(16,kernel_size = 3,strides = 2,padding = 'same')(d3)
    d3 = tfa.layers.InstanceNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    d3 = Conv2DTranspose(8,kernel_size = 3,strides = 1,padding = 'same')(d3)
    output_img = Conv2D(3,kernel_size = 3,strides = 1,padding = 'same',activation='sigmoid')(d3)
    return Model(d0, output_img) 
  def build_label_generator(self):
    d0 = Input(shape=self.img_shape)
    d1 = Conv2D(32,kernel_size = 3,strides = 1,padding = 'same')(d0)
    #d1 = BatchNormalization()(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(32,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(64,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(64,kernel_size = 3,strides = 1,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(64,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(128,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(128,kernel_size = 3,strides = 1,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(128,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(256,kernel_size = 3,strides = 2,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(512,kernel_size = 3,strides = 2,padding = 'same')(d1)
    df2 = Flatten()(d1)
    d2 = Dense(512,activation = 'relu')(df2)
    d2 = Dense(128,activation='relu')(d2)
    d2 = Dense(32,activation='relu')(d2)
    d2 = Dense(1,activation='sigmoid')(d2)
    d3 = Dense(512,activation = 'relu')(df2)
    d3 = Dense(128,activation='relu')(d3)
    d3 = Dense(32,activation='relu')(d3)
    d3 = Dense(1,activation='sigmoid')(d3)
    d4 = Dense(512,activation = 'relu')(df2)
    d4 = Dense(128,activation='relu')(d4)
    d4 = Dense(32,activation='relu')(d4)
    d4 = Dense(1,activation='sigmoid')(d4)
    d5 = Dense(512,activation = 'relu')(df2)
    d5 = Dense(128,activation='relu')(d5)
    d5 = Dense(32,activation='relu')(d5)
    d5 = Dense(1,activation='sigmoid')(d5)
    d6 = Dense(512,activation = 'relu')(df2)
    d6 = Dense(128,activation='relu')(d6)
    d6 = Dense(32,activation='relu')(d6)
    d6 = Dense(1,activation='sigmoid')(d6)
    d7 = Dense(512,activation = 'relu')(df2)
    d7 = Dense(128,activation='relu')(d7)
    d7 = Dense(32,activation='relu')(d7)
    d7 = Dense(1,activation='sigmoid')(d7)
    d8 = Dense(512,activation = 'relu')(df2)
    d8 = Dense(128,activation='relu')(d8)
    d8 = Dense(32,activation='relu')(d8)
    d8 = Dense(1,activation='sigmoid')(d8)
    #output_label = Dense(12,activation='sigmoid')(d2)
    return Model(d0, [d2,d3,d4,d5,d6,d7,d8])
  def build_img_label_discriminator(self):
    x1 = Input(shape=self.img_shape)
    x2 = Input(shape=self.label_shape)
    d1 = Conv2D(32,kernel_size = 3,strides = 4,kernel_constraint=self.const, padding = 'same')(x1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(64,kernel_size = 3,strides = 4,kernel_constraint=self.const,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(128,kernel_size = 3,strides = 2,kernel_constraint=self.const,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(128,kernel_size = 3,strides = 2,kernel_constraint=self.const,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d1 = Conv2D(512,kernel_size = 3,strides = 2,kernel_constraint=self.const,padding = 'same')(d1)
    d1 = tfa.layers.InstanceNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    d2 = Flatten()(d1)
    d4 = Dense(512)(x2)
    d3 = concatenate([d4,d2])
    d3 = LeakyReLU(0.2)(d3)
    d3 = Dense(64)(d3)
    d3 = LeakyReLU(0.2)(d3)
    d3 = Dense(32)(d3)
    d3 = LeakyReLU(0.2)(d3)
    validity = Dense(1)(d3)
    return Model(inputs = [x1,x2], outputs = [validity])
  def train(self, epochs, batch_size=64, sample_interval=50):
    valid = -np.ones((batch_size,1))
    fake = np.ones((batch_size,1))
    for epoch in range(epochs):
      for i in range(5):
        ep, A,B= next(self.traindata)
        fake_A = self.g_BA.predict(B)
        dA_loss_real = self.d_A.train_on_batch([A,B], valid) 
        dA_loss_fake = self.d_A.train_on_batch([fake_A,B], fake)
        dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
        f_B = self.g_AB.predict(A)
        fake_B = np.concatenate(f_B,axis=1)
        dB_loss_real = self.d_B.train_on_batch([A,B], valid)
        dB_loss_fake = self.d_B.train_on_batch([A,fake_B], fake)
        dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
        o1,o2,o3,o4,o5,o6,o7 = [], [], [], [], [], [], []
        for i in B[:]:
          o1.append(i[0])
          o2.append(i[1])
          o3.append(i[2])
          o4.append(i[3])
          o5.append(i[4])
          o6.append(i[5])
          o7.append(i[6])
        o1 = np.array(o1)
        o2 = np.array(o2)
        o3 = np.array(o3)
        o4 = np.array(o4)
        o5 = np.array(o5)
        o6 = np.array(o6)
        o7 = np.array(o7)
        classifier_loss = self.classifier.train_on_batch(A,[o1,o2,o3,o4,o5,o6,o7])
      d_loss = 0.5 * np.add(dA_loss, dB_loss)
      ep, A,B= next(self.traindata)
      o1,o2,o3,o4,o5,o6,o7 = [], [], [], [], [], [], []
      for i in B[:]:
        o1.append(i[0])
        o2.append(i[1])
        o3.append(i[2])
        o4.append(i[3])
        o5.append(i[4])
        o6.append(i[5])
        o7.append(i[6])
      o1 = np.array(o1)
      o2 = np.array(o2)
      o3 = np.array(o3)
      o4 = np.array(o4)
      o5 = np.array(o5)
      o6 = np.array(o6)
      o7 = np.array(o7)
      g_loss = self.combined.train_on_batch([A,B],[valid, valid, A,o1,o2,o3,o4,o5,o6,o7])
      if epoch % sample_interval == 0: 
        self.dloss.append(np.mean(d_loss))
        self.gloss.append(np.mean(g_loss))
        self.iter.append(epoch)
        print("%d [D loss : %f ] [G loss : %f]"%(epoch,np.mean(d_loss),np.mean(g_loss)))
        res = gan.g_AB.predict(np.array([A[0]]))
        lk = [5,10,53,172,47,5,9]
        r,b = [],[]
        for i,v in enumerate(lk):
          p = res[i]*v
          r.append(p[0][0])
          b.append(B[0][i]*v)
        print(b)
        print(np.round(np.array(r)))
      if epoch % 10 == 0:
        self.g_AB.save('/content/drive/MyDrive/g_ABclass')


In [ ]:
gan = CycleGAN(train_batch,const)


In [ ]:
gan.train(epochs=1500, batch_size=64, sample_interval=1)

0 [D loss : -0.000388 ] [G loss : 1.583081]
[1.0, 3.0, 2.0, 6.0, 2.9999999999999996, 2.0, 2.0]
[ 1.  3.  9. 29.  8.  2.  3.]
INFO:tensorflow:Assets written to: /content/drive/MyDrive/g_ABclass/assets
1 [D loss : 0.000843 ] [G loss : 1.552821]
[1.0, 1.0, 4.0, 27.0, 5.0, 2.0, 4.0]
[ 2.  1.  4. 11.  6.  2.  2.]
2 [D loss : -0.000347 ] [G loss : 1.663343]
[1.0, 1.0, 1.0, 44.0, 2.0, 1.0, 1.0]
[1. 1. 3. 7. 6. 2. 1.]
3 [D loss : -0.000498 ] [G loss : 1.525469]
[1.0, 1.0, 1.0, 1.0, 7.0, 2.0, 1.0]
[1. 1. 2. 8. 6. 2. 1.]
4 [D loss : 0.000134 ] [G loss : 1.539349]
[1.0, 1.0, 1.0, 3.0, 2.9999999999999996, 2.0, 1.0]
[1. 1. 1. 6. 5. 2. 2.]
5 [D loss : -0.001195 ] [G loss : 1.508403]
[1.0, 1.0, 4.0, 26.0, 5.0, 2.0, 2.0]
[ 2.  1.  3. 13.  7.  2.  2.]
6 [D loss : -0.001502 ] [G loss : 1.516699]
[1.0, 1.0, 1.0, 1.0, 29.0, 2.0, 1.0]
[1. 1. 2. 7. 6. 1. 1.]
7 [D loss : -0.001714 ] [G loss : 1.557344]
[1.0, 2.0, 29.0, 68.0, 20.0, 1.0, 1.0]
[ 2.  3.  6. 12.  5.  2.  1.]
8 [D loss : -0.002589 ] [G loss : 1.53